# 🎥 ASU RoomSense – Real-Time People Monitoring (MacBook Camera)
Welcome to the live monitoring notebook! 🧠

This notebook captures webcam frames **one at a time**, detects people using a pretrained YOLOv5 model, and logs the count along with timestamps. It’s designed to work with your **MacBook's built-in camera**, not iPhone Continuity Camera.

🔍 You will see annotated images directly below each cell. All data will be stored in a log for later analysis.

## 🔧 Step 1: Import Required Libraries
We load all the necessary Python libraries including OpenCV, PyTorch, Matplotlib, and Pandas.

In [1]:
!pip install opencv-python
import cv2
import torch
import time
import pandas as pd
from IPython.display import clear_output, display
import matplotlib.pyplot as plt

## 🧠 Step 2: Load YOLOv5 Model
We use a lightweight **YOLOv5s** model to detect people. The model is loaded via PyTorch Hub and filtered to detect only the **'person'** class.

In [3]:
# Load YOLOv5 model (first run may download weights)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.classes = [0]  # Class 0 = person

Using cache found in /Users/yashwanthpg/.cache/torch/hub/ultralytics_yolov5_master


requirements: Ultralytics requirement ['setuptools>=70.0.0'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.5.1
    Uninstalling setuptools-69.5.1:
      Successfully uninstalled setuptools-69.5.1

requirements: AutoUpdate success ✅ 4.2s, installed 1 package: ['setuptools>=70.0.0']
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2025-4-27 Python-3.12.4 torch-2.7.0 CPU

100%|██████████████████████████████████████| 14.1M/14.1M [00:01<00:00, 13.9MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


## 🎥 Step 3: Connect to the MacBook Webcam
Here we explicitly choose camera index `1`, which corresponds to your **built-in MacBook webcam** (not iPhone Continuity Camera).

We'll also prepare a log list to store results.

In [140]:
# Connect to internal camera
cap = cv2.VideoCapture(1, cv2.CAP_AVFOUNDATION)  # Use camera index 1 for internal cam
log = []

## 🌀 Step 4: Frame-by-Frame Monitoring
We now loop through 30 frames, detect people in each frame, annotate the result, and display it below.

- ⏱️ Runs one frame every second
- 👥 Counts number of people per frame
- 🧾 Logs timestamp and count
- 📷 Displays the annotated result

In [142]:
for i in range(30):
    ret, frame = cap.read()
    if not ret:
        print("❌ Camera not accessible")
        break

    results = model(frame)
    count = len(results.xyxy[0])
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    log.append({'time': timestamp, 'count': count})

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results.render()

    clear_output(wait=True)
    plt.figure(figsize=(8, 6))
    plt.imshow(results.ims[0])
    plt.axis('off')
    plt.suptitle(f'🖼️ Frame {i+1} of 30', fontsize=16, fontweight='bold')
    plt.title(f'👥 Detected People: {count}   🕒 {timestamp}', fontsize=12)
    display(plt.gcf())
    plt.close()

    # 🔥 Updated print statement
    print(f"📸 Processing Frame {i+1}/30")
    print(f"   📅 Timestamp: {timestamp}")
    print(f"   👤 People Detected: {count}")
    print("-" * 50)

    time.sleep(1)


<Figure size 800x600 with 1 Axes>

📸 Processing Frame 30/30
   📅 Timestamp: 2025-04-27 15:38:24
   👤 People Detected: 2
--------------------------------------------------


In [144]:
# 📸 Save selected frames (1st, 15th, 30th if they exist)
import os
os.makedirs("frames", exist_ok=True)

if len(results.ims) > 0:
    if i == 0:
        plt.imsave("frames/frame_1.png", results.ims[0])
    elif i == 14:
        plt.imsave("frames/frame_15.png", results.ims[0])
    elif i == 29:
        plt.imsave("frames/frame_30.png", results.ims[0])

## 💾 Step 5: Save Log to CSV
After monitoring is complete, we save the timestamped log to a CSV file for analysis later.

## 🗺️ Define Activity Zones
We define specific rectangular zones in the frame (e.g., `entry`, `help_desk`) to track where people are detected.
Each detection is assigned to a zone based on the center of the bounding box.

In [146]:
# Define room zones (pixel coordinates)
zones = {
    'entry': [(0, 0), (250, 480)],
    'help_desk': [(250, 0), (640, 480)]
}

def get_zone(person_box, zones):
    x_center = (person_box[0] + person_box[2]) / 2
    y_center = (person_box[1] + person_box[3]) / 2
    for name, ((x1, y1), (x2, y2)) in zones.items():
        if x1 <= x_center <= x2 and y1 <= y_center <= y2:
            return name
    return 'unknown'

## 🔁 Real-Time Frame Monitoring with Zone Tracking
This loop detects people and logs both their total count and zone-specific counts.

In [148]:
log = []  # reset log
for i in range(30):
    ret, frame = cap.read()
    if not ret:
        print("❌ Camera not accessible")
        break

    results = model(frame)
    boxes = results.xyxy[0].cpu().numpy()
    count = len(boxes)

    zone_counts = {}
    for box in boxes:
        zone = get_zone(box[:4], zones)
        zone_counts[zone] = zone_counts.get(zone, 0) + 1

    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    log.append({'time': timestamp, 'count': count, 'zones': zone_counts})

    results.render()
    clear_output(wait=True)
    plt.figure(figsize=(8, 6))
    plt.imshow(results.ims[0])
    plt.axis('off')
    plt.title(f'📷 Frame {i+1} – 👤 Total People: {count}', fontsize=14, fontweight='bold')
    display(plt.gcf())
    plt.close()

    # 🖨️ Updated print statement:
    print(f"🕒 {timestamp} | Frame {i+1}/30")
    print(f"👥 Total People Detected: {count}")
    print(f"🗺️ Zone Breakdown: {zone_counts}")
    print("=" * 60)

    time.sleep(1)


<Figure size 800x600 with 1 Axes>

🕒 2025-04-27 15:39:09 | Frame 30/30
👥 Total People Detected: 2
🗺️ Zone Breakdown: {'unknown': 1, 'help_desk': 1}


## 💾 Save Enhanced Log (Including Zones)

In [150]:
cap.release()
df = pd.DataFrame(log)
df.to_csv("room_log_with_zones.csv", index=False)
df.head()

,time,count,zones
0,2025-04-27 15:38:36,2,{'unknown': 2}
1,2025-04-27 15:38:38,2,{'unknown': 2}
2,2025-04-27 15:38:39,2,{'unknown': 2}
3,2025-04-27 15:38:40,2,{'unknown': 2}
4,2025-04-27 15:38:41,2,{'unknown': 2}


In [152]:
cap.release()
df = pd.DataFrame(log)
df.to_csv("jupyter_live_log.csv", index=False)
df.head()

,time,count,zones
0,2025-04-27 15:38:36,2,{'unknown': 2}
1,2025-04-27 15:38:38,2,{'unknown': 2}
2,2025-04-27 15:38:39,2,{'unknown': 2}
3,2025-04-27 15:38:40,2,{'unknown': 2}
4,2025-04-27 15:38:41,2,{'unknown': 2}


---
🎉 **Done!** You've just captured and analyzed 30 frames in real time with live annotations. You can now open `jupyter_live_log.csv` for further data analysis or reporting.

💡 Tip: Extend the loop for more frames or add zone tracking to enhance this notebook further!

## 📊 Step 6: Analyze Captured Data
Now that we’ve saved the real-time detection data, let’s load it and visualize how room occupancy changed during our monitoring session.

In [154]:
# Load the CSV file
df = pd.read_csv("jupyter_live_log.csv")
df['time'] = pd.to_datetime(df['time'])
df.head()

,time,count,zones
0,2025-04-27 15:38:36,2,{'unknown': 2}
1,2025-04-27 15:38:38,2,{'unknown': 2}
2,2025-04-27 15:38:39,2,{'unknown': 2}
3,2025-04-27 15:38:40,2,{'unknown': 2}
4,2025-04-27 15:38:41,2,{'unknown': 2}


### 📈 Plot: Occupancy Over Time
This line chart shows the number of people detected in each frame over time.

In [156]:
plt.figure(figsize=(12, 5))
plt.plot(df['time'], df['count'], marker='o', linestyle='-')
plt.title('👥 People Detected Over Time')
plt.xlabel('Time')
plt.ylabel('Number of People')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 📌 Summary Statistics
Let’s display the average, max, and min number of people detected during the session.

In [158]:
summary = df['count'].describe()
summary

count    30.000000
mean      1.900000
std       0.305129
min       1.000000
25%       2.000000
50%       2.000000
75%       2.000000
max       2.000000
Name: count, dtype: float64

---
✅ **Analysis Complete!** You've visualized real-time people tracking data and generated key insights.

You can export this notebook, include it in presentations, or continue by adding zone-based tracking and alerts.

## 🔍 Step 7: Zone-Specific Heatmaps
Let's extract and visualize how frequently different zones were used across all captured frames.

We assume each detection in the original video frame was tagged with the zone it appeared in (e.g., `entry`, `help_desk`).

In [160]:
import ast
# Add synthetic example if real zones aren't logged
# Convert dictionary-style strings into actual Python dicts if zone log exists
try:
    df = pd.read_csv("jupyter_live_log.csv")
    df['time'] = pd.to_datetime(df['time'])
    df['zones'] = df['zones'].apply(ast.literal_eval)
    zone_df = df['zones'].apply(pd.Series).fillna(0)
    zone_totals = zone_df.sum()
except:
    print("No zone data available. Skipping zone-specific analysis.")

### 📊 Heatmap of Zone Activity
This heatmap shows which areas of the room were most frequently active.

In [162]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
sns.barplot(x=zone_totals.index, y=zone_totals.values, hue=zone_totals.index, palette="coolwarm", dodge=False, legend=False)

plt.title("🔥 Zone Usage Frequency")
plt.ylabel("# of People Detected")
plt.xlabel("Zone")
plt.show()

## 🚨 Step 8: Real-Time Alerts Summary
We simulate a live alert system by identifying all time points where total room occupancy exceeded a set threshold (e.g., 5 people).

In [164]:
THRESHOLD = 5
over_threshold = df[df['count'] > THRESHOLD]
print(f"⚠️ {len(over_threshold)} alerts triggered. Here are the timestamps:")
over_threshold[['time', 'count']]

⚠️ 0 alerts triggered. Here are the timestamps:


,time,count


## ✅ Final Insights
This notebook completes the analytical cycle described in your CIS 515 proposal:
- 🔎 **Detection & Tracking** via webcam + YOLO
- 🧠 **Zone Monitoring** across room spaces
- 📊 **Statistical & Visual Analysis** of room activity
- 🚨 **Real-Time Alerting** for overcrowded conditions

**Next steps:** Deploying this into an interactive Streamlit dashboard for live visualization and broader campus utility.

##🖼️ Sample Captured Frames (Annotated)
Here are a few example annotated frames saved during live detection:

![Frame 1](frames/frame_1.png)
![Frame 15](frames/frame_15.png)
![Frame 30](frames/frame_30.png)